<a href="https://colab.research.google.com/github/stevegabriel1/spark_nlp/blob/main/spark_nlp_session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This is following directly the ODSC East 2021 session 'Introduction to Spark NLP' from John Snow Labs' Veysel Kocaman.**

In [1]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
!bash colab_setup.sh

# Install sparknlp-display
! pip install spark-nlp-display

--2021-04-24 08:12:08--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1594 (1.6K) [text/plain]
Saving to: ‘colab_setup.sh’

colab_setup.sh      100%[===================>]   1.56K  --.-KB/s    in 0s      

2021-04-24 08:12:08 (31.7 MB/s) - ‘colab_setup.sh’ saved [1594/1594]

setup Colab for PySpark 3.0.2 and Spark NLP 3.0.2
     |████████████████████████████████| 204.8MB 72kB/s 
     |████████████████████████████████| 153kB 19.9MB/s 
     |████████████████████████████████| 204kB 21.7MB/s 
     |████████████████████████████████| 102kB 4.1MB/s 
     |████████████████████████████████| 71kB 5.1MB/s 


In [2]:
import sparknlp
from pyspark.ml import Pipeline
from sparknlp.base import *
import pyspark.sql.functions as F
from sparknlp.annotator import *

spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 3.0.2
Apache Spark version: 3.0.2


## Using Pretrained Pipelines

for a more detailed notebook, see https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/1.SparkNLP_Basics.ipynb

In [3]:
from sparknlp.pretrained import PretrainedPipeline

In [4]:
pipeline_dl = PretrainedPipeline('explain_document_dl', lang='en')

explain_document_dl download started this may take some time.
Approx size to download 169.3 MB
[OK!]


**Stages**
- DocumentAssembler
- SentenceDetector
- Tokenizer
- NER (NER with GloVe 100D embeddings, CoNLL2003 dataset)
- Lemmatizer
- Stemmer
- Part of Speech
- SpellChecker (Norvig)


In [5]:
testDoc = '''
Blue and green should be seen.
It seems that Coca Cola Amitil will be bought out very soon.
Do you prefer sparkling or regular minerall water?
Their vehicle was quick off the mark compared to most other cars.
The tram network in Melbourne is extensive.
'''

result = pipeline_dl.annotate(testDoc)


In [6]:
result.keys()

dict_keys(['entities', 'stem', 'checked', 'lemma', 'document', 'pos', 'token', 'ner', 'embeddings', 'sentence'])

In [7]:
result['entities']

['Blue', 'Coca Cola Amitil', 'Melbourne']

In [8]:
import pandas as pd

df = pd.DataFrame({'token':result['token'], 'ner_label':result['ner'],
                      'spell_corrected':result['checked'], 'POS':result['pos'],
                      'lemmas':result['lemma'], 'stems':result['stem']})

df

,token,ner_label,spell_corrected,POS,lemmas,stems
0,Blue,B-ORG,Blue,NNP,Blue,blue
1,and,O,and,CC,and,and
2,green,O,green,NN,green,green
3,should,O,should,MD,should,should
4,be,O,be,VB,be,be
5,seen,O,seen,VBN,see,seen
6,.,O,.,.,.,.
7,It,O,It,PRP,It,it
8,seems,O,seems,VBZ,seem,seem
9,that,O,that,IN,that,that


In [9]:
detailed_result = pipeline_dl.fullAnnotate(testDoc)

detailed_result[0]['entities']

[Annotation(chunk, 1, 4, Blue, {'entity': 'ORG', 'sentence': '0', 'chunk': '0'}),
 Annotation(chunk, 46, 61, Coca Cola Amitil, {'entity': 'ORG', 'sentence': '1', 'chunk': '1'}),
 Annotation(chunk, 230, 238, Melbourne, {'entity': 'LOC', 'sentence': '4', 'chunk': '2'})]

In [10]:
chunks=[]
entities=[]
for n in detailed_result[0]['entities']:
        
  chunks.append(n.result)
  entities.append(n.metadata['entity']) 
    
df = pd.DataFrame({'chunks':chunks, 'entities':entities})
df    

,chunks,entities
0,Blue,ORG
1,Coca Cola Amitil,ORG
2,Melbourne,LOC


In [11]:
tuples = []

for x,y,z in zip(detailed_result[0]["token"], detailed_result[0]["pos"], detailed_result[0]["ner"]):

  tuples.append((int(x.metadata['sentence']), x.result, x.begin, x.end, y.result, z.result))

df = pd.DataFrame(tuples, columns=['sent_id','token','start','end','pos', 'ner'])

df


,sent_id,token,start,end,pos,ner
0,0,Blue,1,4,NNP,B-ORG
1,0,and,6,8,CC,O
2,0,green,10,14,NN,O
3,0,should,16,21,MD,O
4,0,be,23,24,VB,O
5,0,seen,26,29,VBN,O
6,0,.,30,30,.,O
7,1,It,32,33,PRP,O
8,1,seems,35,39,VBZ,O
9,1,that,41,44,IN,O


### Spell Checker DL

https://medium.com/spark-nlp/applying-context-aware-spell-checking-in-spark-nlp-3c29c46963bc

#### Creating a Pretrained Pipeline From Scratch
Let's start with building a pipeline; a *spell correction pipeline*. We will use a pretrained model from our library.

In [12]:
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

from IPython.utils.text import columnize

In [13]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")\
    
spellModel = ContextSpellCheckerModel\
    .pretrained('spellcheck_dl')\
    .setInputCols("token")\
    .setOutputCol("checked")\
    .setErrorThreshold(4.0)\
    .setTradeoff(6.0)

finisher = Finisher()\
    .setInputCols("checked")

pipeline = Pipeline(
    stages = [
    documentAssembler,
    tokenizer,
    spellModel,
    finisher
  ])

empty_df = spark.createDataFrame([['']]).toDF("text")

spell_pipeline = pipeline.fit(empty_df)

spellcheck_dl download started this may take some time.
Approximate size to download 111.4 MB
[OK!]


In [14]:
spell_pipeline.stages

[DocumentAssembler_5b18dc116287,
 REGEX_TOKENIZER_99549159057f,
 SPELL_b22681bc00ec,
 Finisher_68ef48e732c4]

In [15]:
spell_pipeline.save('SavedSpellChecker')

In [16]:
!ls -lt

total 215340
drwxr-xr-x  4 root root      4096 Apr 24 08:19 SavedSpellChecker
-rw-r--r--  1 root root      1594 Apr 24 08:12 colab_setup.sh
drwxr-xr-x  1 root root      4096 Apr 21 13:39 sample_data
-rw-r--r--  1 root root 220488957 Feb 16 06:33 spark-3.0.2-bin-hadoop2.7.tgz
drwxr-xr-x 13 1000 1000      4096 Feb 16 05:21 spark-3.0.2-bin-hadoop2.7


In [17]:
pipeline_local = PretrainedPipeline.from_disk('SavedSpellChecker')

**Note: I have changed the second sentence in testDoc below, with the mispelt 'bycycle' included.**

In [18]:
testDoc = '''
During the summer we usually have the best ueather."
I have a green bycycle in the garage, so nice."
'''

result = pipeline_local.annotate(testDoc)

In [19]:
result

{'checked': ['During',
  'the',
  'summer',
  'we',
  'usually',
  'have',
  'the',
  'best',
  'leather',
  '."',
  ',',
  'have',
  ',',
  'green',
  'bicycle',
  'in',
  'the',
  'garage',
  ',',
  'so',
  'nice',
  '."']}

### Text Matcher

In [21]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv

news_df = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")


In [23]:
news_df.show(7, truncate=50)

+--------+--------------------------------------------------+
|category|                                       description|
+--------+--------------------------------------------------+
|Business| Short sellers, Wall Street's dwindling band of...|
|Business| Private investment firm Carlyle Group, which h...|
|Business| Soaring crude prices plus worries about the ec...|
|Business| Authorities have halted oil export flows from ...|
|Business| Tearaway world oil prices, toppling records an...|
|Business| Stocks ended slightly higher on Friday but sta...|
|Business| Assets of the nation's retail money market mut...|
+--------+--------------------------------------------------+
only showing top 7 rows

